# Test - CNN classifier
Network Communication LAB Week5 USRP (3) Experiment

Colab의 CUDA와 비교하기 위해서 생성해준 Py model

#### import Library / Set the Data loader / Set Model

In [22]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
import sys
import cv2
import re
import imageio.v2 as imageio
import copy

In [26]:
class CustomDataSet(torch.utils.data.Dataset):

    def __init__(self, data_root):
        self.data_root = data_root
        self.data = []
        self.y = []

        self.DirFolder = os.listdir(self.data_root)
    
        start=datetime.now()
        for FolderIndex in range(len(self.DirFolder)):
            self.DirData = os.listdir(self.data_root + '/' + self.DirFolder[FolderIndex])
            for DataIndex in range(len(self.DirData)):
                img= imageio.imread(self.data_root + '/'  + self.DirFolder[FolderIndex] +'/'+  self.DirData[DataIndex])
                numbers = int(re.sub(r'[^0-9]', '', self.DirFolder[FolderIndex]))
                y1 = int(numbers)
                self.data.append(img)
                self.y.append(y1)
            
        self.data = torch.from_numpy(np.array(self.data))
        self.y = torch.from_numpy(np.array(self.y))
        print('Data reading time spent : ', datetime.now()-start)


    def __getitem__(self, index):
        return self.data[index], self.y[index]
    
    def __len__(self):
        return len(self.data)
        

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class MyClassifier(nn.Module):

    def __init__(self):
        super(MyClassifier,self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 12 outputs
        self.fc2 = torch.nn.Linear(625, 12, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

model = MyClassifier()

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.layer2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.linear1 = nn.Linear(2048, 1024)
        self.linear2 = nn.Linear(1024, 16)

    def forward(self, x):
        #print(x.size()) # Check data size
        x = F.max_pool2d(F.relu(self.layer1(x)), (2,2))
        #print(x.size())
        x = F.max_pool2d(F.relu(self.layer2(x)), (2,2))
        #print(x.size())
        x = torch.flatten(x,1)
        #print(x.size())
        x = F.relu(self.linear1(x))
        #print(x.size())
        x = self.linear2(x)
        #print(x.size())
        return x

model = MyClassifier()

print(model)

td = torch.rand(50,1,64,64)
out = model(td)
print(out.size())

MyClassifier(
  (layer1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (linear1): Linear(in_features=2048, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=16, bias=True)
)
torch.Size([50, 16])


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.layer2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.layer3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 2, padding = 1)
        self.linear1 = nn.Linear(4096, 1024)
        self.linear2 = nn.Linear(1024, 16)

    def forward(self, x):
        #print(x.size()) # Check data size
        x = F.max_pool2d(F.relu(self.layer1(x)), (2,2))
        #print(x.size())
        x = F.max_pool2d(F.relu(self.layer2(x)), (2,2))
        #print(x.size())
        x = F.max_pool2d(F.relu(self.layer3(x)), (2,2))
        x = torch.flatten(x,1)
        #print(x.size())
        x = F.relu(self.linear1(x))
        #print(x.size())
        x = self.linear2(x)
        #print(x.size())
        return x

model = MyClassifier()

print(model)

td = torch.rand(50,1,64,64)
out = model(td)
print(out.size())

MyClassifier(
  (layer1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (layer3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (linear1): Linear(in_features=4096, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=16, bias=True)
)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x256 and 4096x1024)

In [5]:
from torchviz import make_dot
from torch.autograd import Variable
td = torch.rand(50,1,64,64)
make_dot(model(td), params = dict(model.named_parameters())).render('graph', format='png')

'graph.png'

In [6]:
!pip install torchsummary

In [2]:
from torchsummary import summary
summary(model,input_size=(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             416
            Conv2d-2           [-1, 32, 16, 16]           4,640
            Linear-3                 [-1, 1024]       2,098,176
            Linear-4                   [-1, 16]          16,400
Total params: 2,119,632
Trainable params: 2,119,632
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.57
Params size (MB): 8.09
Estimated Total Size (MB): 8.67
----------------------------------------------------------------


### Experiment 01 
---
* given label dataset과 given model을 이용해서 실험 
* 총 10회를 반복해서 그중에서 best model 의 pth 로 저장하고 저장된 모델을 이용해서 앞서 test  모델의 실험다음으로 진행하는  
생성한 label dataset에 대해서도 test를 진행하고자 한다.


* Input data : labeled_data 
* Output : Save Experiment 01 

In [33]:
def main():
    # # GPU Allocating
    print('==> CPU/GPU allocating..')
    print(torch.backends.mps.is_available())
    # device = torch.device("cuda")
    device = torch.device("mps")
    print( 'Device : ', device)
    # print ('Available devices : ', torch.device_count())
    # print('Selecing GPU : ',torch.get_device_name(device))
    total_t = datetime.now()

    # Load Data
    print('==> Dataset selecting..')
    data_root = "./labeled_data" # Set your data root
    # data_root = "./labeled_data_self" # Set your data root
    MyDataSet = CustomDataSet(data_root)

    # Data Set Split
    train_size = int(0.7 * len(MyDataSet)) # Training Size
    valid_size = int(0.2 * len(MyDataSet)) # Validation Size
    test_size = len(MyDataSet) - train_size - valid_size # Test Size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(MyDataSet, [train_size, valid_size, test_size])

    print('Number of train data : ', len(train_dataset))
    print('Number of valid data : ', len(valid_dataset))
    print('Number of test data : ', len(test_dataset))

    # Model
    print('==> Building model..')
    nb_epochs = 30 # Set an epoch 1
    torch.manual_seed(10) #Set your seed number
    BatchSize = 50 # Set a batch size

    resume = False #If you have the training networks, change this to True.
    LoadPath = './save'
    LoadEpoch = 14 #Set your best model epoch.
    if resume == True: 
        LoadName = 'best_model_'+ str(LoadEpoch) + '.pth'
        model = torch.load(os.path.join(LoadPath,LoadName))
    else:
        model = MyClassifier()
        LoadEpoch = 0
    
    model = model.to(device)
    print('Is resume : ', resume)
    print('Number of epochs : {0}, Batch size: {1}'.format(nb_epochs, BatchSize))

    if device == 'cuda':
        #model = torch.nn.DataParallel(model)
        cudnn.benchmark = True

    train_dataloader = DataLoader(train_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
    test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers=0)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader), eta_min=0, last_epoch=-1)

    # Start the log
    if os.path.isdir(LoadPath) == False:
        os.mkdir(LoadPath)

    original_stdout = sys.stdout
    if resume == False:
        with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','w') as f:
        # with open(LoadPath + '/result.txt','w') as f:
            sys.stdout = f
            print(sys._getframe().f_code.co_filename)
            print(data_root)
            print('start time : ', datetime.now())
            sys.stdout = original_stdout

    min_loss = 99999 # Allocate default value
    print('==> Training model..')
    for epoch in range(1,nb_epochs+1) :
        #Train
        start_epoch = datetime.now()
        model.train()
        train_loss = 0
        valid_total = 0
        valid_correct = 0
        #label_total = list(0. for i in range(16))
        #label_correct = list(0. for i in range(16))
        for batch_idx, (datum, targets) in enumerate(train_dataloader):
            start = datetime.now()
            datum, targets= datum.to(device), targets.long().to(device)
            img = datum.reshape([-1, 1, 64, 64]).float()
            bsz = targets.shape[0]

            # Computing loss
            out = model(img) 
            loss = criterion(out, targets)
            
            # Update model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()
            
            #print('Epoch: {0} | Progressing: {1} / {2} | Train loss: {3:0.6f} | Time spent: {4}, \n'.format(epoch, (batch_idx)*BatchSize + datum[0].size(0), len(train_dataset), train_loss/(batch_idx+1), datetime.now()-start))
            

        #Valid
        model.eval()
        valid_loss_value = 0
        with torch.no_grad():
            for valid_batch_idx, (datum, targets) in enumerate(valid_dataloader):
                datum, targets= datum.to(device), targets.to(device)
                img = datum.reshape([-1, 1, 64, 64]).float()
                bsz = targets.shape[0]
                #computing loss
                out = model(img) 
                _, predicted = torch.max(out,1)
                c = (predicted == targets).squeeze()
                valid_total += targets.size(0)
                valid_correct += (predicted == targets).sum().item()
                #for i in range(bsz):
                #    label = targets[i]
                #    label_correct[label] += c[i].item()
                #    label_total[label] += 1
            
                # compute loss
                valid_loss = criterion(out, targets.long())
                valid_loss_value += valid_loss.item()

        # remember minimum loss model
        is_min = valid_loss_value < min_loss
        min_loss = min(valid_loss_value, min_loss)

        if is_min == True:
            best_model = copy.deepcopy(model)
            best_epoch = epoch

        print('Epoch: {0} | Train loss: {1:0.6f}, Valid loss: {2:0.6f}, Valid Acc: {3} / {4} | Time spent: {5}'.format(epoch, train_loss/(batch_idx+1), valid_loss_value/(valid_batch_idx+1), valid_correct, valid_total, datetime.now()-start_epoch ))
        # Save the log
        with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','a') as f:
        # with open(LoadPath+'/result.txt','a') as f:
            sys.stdout = f
            print('Epoch: {0} | Train loss: {1:0.6f}, Valid loss: {2:0.6f}, Valid Acc: {3} / {4} | Time spent: {5}'.format(epoch, train_loss/(batch_idx+1), valid_loss_value/(valid_batch_idx+1), valid_correct, valid_total, datetime.now()-start_epoch ))
            sys.stdout = original_stdout
            

    #Test
    print('==> Testing the model..')
    best_model.eval()
    test_loss_value = 0
    total = 0
    correct = 0
    label_total = list(0. for i in range(16))
    label_correct = list(0. for i in range(16))
    with torch.no_grad():
        for test_batch_idx, (datum, targets) in enumerate(test_dataloader):
            datum, targets= datum.to(device), targets.to(device)
            img = datum.reshape([-1, 1, 64, 64]).float()
            bsz = targets.shape[0]
            #computing loss
            out = best_model(img) 
            _, predicted = torch.max(out,1)
            c = (predicted == targets).squeeze()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            for i in range(bsz):
                label = targets[i]
                label_correct[label] += c[i].item()
                label_total[label] += 1
        
            # compute loss
            test_loss = criterion(out, targets.long())
            test_loss_value += test_loss.item()
    

    # Save the best model
    SaveName = 'best_model_'+ str(LoadEpoch + best_epoch) + '.pth'
    label_total = list(0. for i in range(16))
    torch.save(best_model, os.path.join(LoadPath,SaveName))

    print('Best epoch : ', best_epoch)    
    for i in range(14):
        print('Accuracy of label {0} : {1:0.0f} / {2:0.0f}'.format(i, label_correct[i], label_total[i]))

    print('Test Accuracy : {0:0.3f}, {1} / {2}'.format(correct/total * 100, correct, total ) )

    # Save the log
    with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','a') as f:
        sys.stdout = f
        print('Best epoch : ', best_epoch)    
        for i in range(16):
            print('Accuracy of label {0} : {1:0.0f} / {2:0.0f}'.format(i, label_correct[i], label_total[i]))
        print('Test Accuracy : {0:0.3f}%, ({1} / {2})'.format(correct/total * 100, correct, total ) )

        result_accuracy = round(correct / total * 100, 3)

        sys.stdout = original_stdout
    
    # return result_accuracy, correct, total

if __name__ == '__main__':
    # Result_accuracy = []
    # Result_correct = []
    # Result_total = []
    for repeat_test in range(1):
        main()
    #     Result_accuracy.append(main()[0])
    #     Result_correct.append(main()[1])
    #     Result_total.append(main()[2])
    
    # for i in range(10):
    #     print('Test Accuracy : {0:0.3f}%, ({1} / {2})'.format(Result_correct[i]/Result_total[i] * 100, Result_correct[i], Result_total[i]))

==> CPU/GPU allocating..
True
Device :  mps
==> Dataset selecting..
Data reading time spent :  0:00:00.419625
Number of train data :  1680
Number of valid data :  480
Number of test data :  240
==> Building model..
Is resume :  False
Number of epochs : 30, Batch size: 50
==> Training model..


: 

: 

## Take Setting in Tensor Board

[이거 참고해서 만드는중](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html#)

### 1. Tensor Board Setup

In [8]:
from torch.utils.tensorboard import SummaryWriter

In [9]:
# Optimizer, Criterion 설정해 주기 
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader), eta_min=0, last_epoch=-1)

writer = SummaryWriter('runs/experiment_01')

### 2. Data Loader 

In [10]:
# imports
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

nb_epochs = 30 # Set an epoch 1
torch.manual_seed(10) #Set your seed number
BatchSize = 50 # Set a batch size

# Load Data
print('==> Dataset selecting..')
data_root = "./labeled_data" # Set your data root
# data_root = "./labeled_data_self" # Set your data root
MyDataSet = CustomDataSet(data_root)

# Data Set Split
train_size = int(0.7 * len(MyDataSet)) # Training Size
valid_size = int(0.2 * len(MyDataSet)) # Validation Size
test_size = len(MyDataSet) - train_size - valid_size # Test Size
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(MyDataSet, [train_size, valid_size, test_size])

# Dataloader
train_dataloader = DataLoader(train_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers=0)

==> Dataset selecting..
Data reading time spent :  0:00:00.707463


### 3. Writing to TensorBoard

In [11]:
# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [12]:
# get some random training images
import matplotlib
from more_itertools import one

dataiter = iter(train_dataloader)
images, labels = next(dataiter)

images = images.reshape([-1, 1, 64, 64]).float()
# Create grid of images
img_grid = torchvision.utils.make_grid(images)

#show images
# matplotlib_imshow(img_grid, one_channel= True)

# Write to Tensorboard
writer.add_image('Spectogram item', img_grid)

In [13]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

I1011 18:15:59.153190 6222524416 plugin.py:429] Monitor runs begin
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [17]:
tensorboard dev upload --logdir {'00_new_environment/notebook/runs/experiment_01'}

TensorFlow installation not found - running with reduced feature set.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

00_new_environment/notebook/runs/experiment_01

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) ^C
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/DL-torch/bin/tensorboard", line 10, in <module>
    sys.exit(run_main())
  File "/opt/homebrew/Caskroom/miniforge/base/envs/DL-torch/lib/python3.8/site-packages/tensorboard/main.py", line 46, in run_m

In [18]:
writer.add_graph(model, images)
writer.close()

In [19]:
# helper function
def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

# select random images and their target indices
images, labels = select_n_random(MyDataSet.data, MyDataSet.y)

classes = list(0. for i in range(16))

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]
# log embeddings
features = images.view(-1, 64 * 64)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))
writer.close()

### Experiment 03
---
* week4 실험으로부터 생성한 spectogram을 이용해서 만들어준 label dataset과 given model을 이용해서 실험 
* 총 10회를 반복해서 그중에서 best model 의 pth 로 저장하고 저장된 모델을 이용해서 앞서 test  모델의 실험다음으로 진행하는  
생성한 label dataset에 대해서도 test를 진행하고자 한다.


* Input data : labeled_data_self 
* Output : Save Experiment 03

In [7]:
def main():
    # # GPU Allocating
    print('==> CPU/GPU allocating..')
    print(torch.backends.mps.is_available())
    # device = torch.device("cuda")
    device = torch.device("mps")
    print( 'Device : ', device)
    # print ('Available devices : ', torch.device_count())
    # print('Selecing GPU : ',torch.get_device_name(device))
    total_t = datetime.now()

    # Load Data
    print('==> Dataset selecting..')
    # data_root = "./labeled_data" # Set your data root
    data_root = "./labeled_data_self" # Set your data root
    MyDataSet = CustomDataSet(data_root)

    # Data Set Split
    train_size = int(0.7 * len(MyDataSet)) # Training Size
    valid_size = int(0.2 * len(MyDataSet)) # Validation Size
    test_size = len(MyDataSet) - train_size - valid_size # Test Size
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(MyDataSet, [train_size, valid_size, test_size])

    print('Number of train data : ', len(train_dataset))
    print('Number of valid data : ', len(valid_dataset))
    print('Number of test data : ', len(test_dataset))

    # Model
    print('==> Building model..')
    nb_epochs = 30 # Set an epoch 1
    torch.manual_seed(10) #Set your seed number
    BatchSize = 50 # Set a batch size

    resume = False #If you have the training networks, change this to True.
    LoadPath = './save'
    LoadEpoch = 14 #Set your best model epoch.
    if resume == True: 
        LoadName = 'best_model_'+ str(LoadEpoch) + '.pth'
        model = torch.load(os.path.join(LoadPath,LoadName))
    else:
        model = MyClassifier()
        LoadEpoch = 0
    
    model = model.to(device)
    print('Is resume : ', resume)
    print('Number of epochs : {0}, Batch size: {1}'.format(nb_epochs, BatchSize))

    if device == 'cuda':
        #model = torch.nn.DataParallel(model)
        cudnn.benchmark = True


    train_dataloader = DataLoader(train_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BatchSize, shuffle=True, num_workers=0, pin_memory=True)
    test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers=0)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader), eta_min=0,
                                                               last_epoch=-1)

    # Start the log
    if os.path.isdir(LoadPath) == False:
        os.mkdir(LoadPath)

    original_stdout = sys.stdout
    if resume == False:
        with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','w') as f:
        # with open(LoadPath + '/result.txt','w') as f:
            sys.stdout = f
            print(sys._getframe().f_code.co_filename)
            print(data_root)
            print('start time : ', datetime.now())
            sys.stdout = original_stdout

    min_loss = 99999 # Allocate default value
    print('==> Training model..')
    for epoch in range(1,nb_epochs+1) :
        #Train
        start_epoch = datetime.now()
        model.train()
        train_loss = 0
        valid_total = 0
        valid_correct = 0
        #label_total = list(0. for i in range(16))
        #label_correct = list(0. for i in range(16))
        for batch_idx, (datum, targets) in enumerate(train_dataloader):
            start = datetime.now()
            datum, targets= datum.to(device), targets.long().to(device)
            img = datum.reshape([-1, 1, 64, 64]).float()
            bsz = targets.shape[0]

            # Computing loss
            out = model(img) 
            loss = criterion(out, targets)
            
            # Update model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()
            
            #print('Epoch: {0} | Progressing: {1} / {2} | Train loss: {3:0.6f} | Time spent: {4}, \n'.format(epoch, (batch_idx)*BatchSize + datum[0].size(0), len(train_dataset), train_loss/(batch_idx+1), datetime.now()-start))
            

        #Valid
        model.eval()
        valid_loss_value = 0
        with torch.no_grad():
            for valid_batch_idx, (datum, targets) in enumerate(valid_dataloader):
                datum, targets= datum.to(device), targets.to(device)
                img = datum.reshape([-1, 1, 64, 64]).float()
                bsz = targets.shape[0]
                #computing loss
                out = model(img) 
                _, predicted = torch.max(out,1)
                c = (predicted == targets).squeeze()
                valid_total += targets.size(0)
                valid_correct += (predicted == targets).sum().item()
                #for i in range(bsz):
                #    label = targets[i]
                #    label_correct[label] += c[i].item()
                #    label_total[label] += 1
            
                # compute loss
                valid_loss = criterion(out, targets.long())
                valid_loss_value += valid_loss.item()

        # remember minimum loss model
        is_min = valid_loss_value < min_loss
        min_loss = min(valid_loss_value, min_loss)

        if is_min == True:
            best_model = copy.deepcopy(model)
            best_epoch = epoch

        print('Epoch: {0} | Train loss: {1:0.6f}, Valid loss: {2:0.6f}, Valid Acc: {3} / {4} | Time spent: {5}'.format(epoch, train_loss/(batch_idx+1), valid_loss_value/(valid_batch_idx+1), valid_correct, valid_total, datetime.now()-start_epoch ))
        # Save the log
        with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','a') as f:
        # with open(LoadPath+'/result.txt','a') as f:
            sys.stdout = f
            print('Epoch: {0} | Train loss: {1:0.6f}, Valid loss: {2:0.6f}, Valid Acc: {3} / {4} | Time spent: {5}'.format(epoch, train_loss/(batch_idx+1), valid_loss_value/(valid_batch_idx+1), valid_correct, valid_total, datetime.now()-start_epoch ))
            sys.stdout = original_stdout
            

    #Test
    print('==> Testing the model..')
    best_model.eval()
    test_loss_value = 0
    total = 0
    correct = 0
    label_total = list(0. for i in range(16))
    label_correct = list(0. for i in range(16))
    with torch.no_grad():
        for test_batch_idx, (datum, targets) in enumerate(test_dataloader):
            datum, targets= datum.to(device), targets.to(device)
            img = datum.reshape([-1, 1, 64, 64]).float()
            bsz = targets.shape[0]
            #computing loss
            out = best_model(img) 
            _, predicted = torch.max(out,1)
            c = (predicted == targets).squeeze()
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            for i in range(bsz):
                label = targets[i]
                label_correct[label] += c[i].item()
                label_total[label] += 1
        
            # compute loss
            test_loss = criterion(out, targets.long())
            test_loss_value += test_loss.item()
    

    # Save the best model
    SaveName = 'best_model_'+ str(LoadEpoch + best_epoch) + '.pth'
    torch.save(best_model, os.path.join(LoadPath,SaveName))

    print('Best epoch : ', best_epoch)    
    for i in range(14):
        print('Accuracy of label {0} : {1:0.0f} / {2:0.0f}'.format(i, label_correct[i], label_total[i]))

    print('Test Accuracy : {0:0.3f}, {1} / {2}'.format(correct/total * 100, correct, total ) )

    # Save the log
    with open(LoadPath+'/result_Experiment_01_#'+ str(repeat_test)+'.txt','a') as f:
        sys.stdout = f
        print('Best epoch : ', best_epoch)    
        for i in range(16):
            print('Accuracy of label {0} : {1:0.0f} / {2:0.0f}'.format(i, label_correct[i], label_total[i]))
        print('Test Accuracy : {0:0.3f}%, ({1} / {2})'.format(correct/total * 100, correct, total ) )

        sys.stdout = original_stdout

if __name__ == '__main__':
    for repeat_test in range(10):
        main()

==> CPU/GPU allocating..
True
Device :  mps
==> Dataset selecting..
Data reading time spent :  0:00:00.740929
Number of train data :  2116
Number of valid data :  604
Number of test data :  304
==> Building model..
Is resume :  False
Number of epochs : 30, Batch size: 50
==> Training model..
Epoch: 1 | Train loss: inf, Valid loss: 2.275521, Valid Acc: 136 / 604 | Time spent: 0:00:00.858083
Epoch: 2 | Train loss: 1.647866, Valid loss: 0.748470, Valid Acc: 435 / 604 | Time spent: 0:00:00.404371
Epoch: 3 | Train loss: 0.577784, Valid loss: 0.429748, Valid Acc: 500 / 604 | Time spent: 0:00:00.392128
Epoch: 4 | Train loss: 0.366743, Valid loss: 0.493703, Valid Acc: 497 / 604 | Time spent: 0:00:00.370998
Epoch: 5 | Train loss: 0.306355, Valid loss: 0.317051, Valid Acc: 531 / 604 | Time spent: 0:00:00.371688
Epoch: 6 | Train loss: 0.185503, Valid loss: 0.378676, Valid Acc: 526 / 604 | Time spent: 0:00:00.380390
Epoch: 7 | Train loss: 0.187899, Valid loss: 0.274455, Valid Acc: 539 / 604 | Time

## Bonus - 01 General Data Test



### Experiment 01  Model : Given Dataset (labeled_data) --> Test : Custom Dataset (labeled_data2)

In [31]:
model = torch.load('./save_Experiment_01/best_model_Experiment_01.pth')

In [30]:
device = torch.device("mps")
Mydataset = CustomDataSet("./labeled_data2/")
Mydataloader = DataLoader(Mydataset, batch_size=len(Mydataset), shuffle= True, num_workers=0)

#Test
print('==> Testing the model..')
model.eval()
test_loss_value = 0
total = 0
correct = 0
label_total = list(0. for i in range(16))
label_correct = list(0. for i in range(16))
with torch.no_grad():
    for test_batch_idx, (datum, targets) in enumerate(Mydataloader):
        datum, targets= datum.to(device), targets.to(device)
        img = datum.reshape([-1, 1, 64, 64]).float()
        bsz = targets.shape[0]
        #computing loss
        out = model(img) 
        _, predicted = torch.max(out,1)
        c = (predicted == targets).squeeze()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        for i in range(bsz):
            label = targets[i]
            label_correct[label] += c[i].item()
            label_total[label] += 1
        # compute loss
        # test_loss = criterion(out, targets.long())
        # test_loss_value += test_loss.item()

# Test for label 1, 4, 5 only
label_145 = [1,4,5]
correct_145, total_145 = 0,0
# for i in range(16):
for i in label_145:
    try:    
        correct_145 += label_correct[i]
        total_145 += label_total[i]
        print('Accuracy of label {0} : {1:0.0f} / {2:0.0f} -> {3:0.3f}'.format(i, label_correct[i], label_total[i], label_correct[i]/label_total[i] * 100))
    except ZeroDivisionError:
        print("Zero Division Error -> since there are no images in label folder")
print('Test Accuracy of label 1,4,5 : {0:0.3f}, {1:0.0f} / {2:0.0f}'.format(correct_145/total_145 * 100, correct_145, total_145 ) )
    

Data reading time spent :  0:00:00.719603
==> Testing the model..
Accuracy of label 1 : 373 / 374 -> 99.733
Accuracy of label 4 : 372 / 375 -> 99.200
Accuracy of label 5 : 472 / 475 -> 99.368
Test Accuracy of label 1,4,5 : 99.428, 1217 / 1224


### Experiment 02  Model : Custom Dataset (labeled_data2)  --> Test : Given Dataset (labeled_data)

In [29]:
model = torch.load('./save_Experiment_03/best_model_Experiment_03.pth')

In [32]:
device = torch.device("mps")
Mydataset = CustomDataSet("./labeled_data/")
Mydataloader = DataLoader(Mydataset, batch_size=len(Mydataset), shuffle= True, num_workers=0)

#Test
print('==> Testing the model..')
model.eval()
test_loss_value = 0
total = 0
correct = 0
label_total = list(0. for i in range(16))
label_correct = list(0. for i in range(16))
with torch.no_grad():
    for test_batch_idx, (datum, targets) in enumerate(Mydataloader):
        datum, targets= datum.to(device), targets.to(device)
        img = datum.reshape([-1, 1, 64, 64]).float()
        bsz = targets.shape[0]
        #computing loss
        out = model(img) 
        _, predicted = torch.max(out,1)

        c = (predicted == targets).squeeze()
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        for i in range(bsz):
            label = targets[i]
            label_correct[label] += c[i].item()
            label_total[label] += 1
        # compute loss
        # test_loss = criterion(out, targets.long())
        # test_loss_value += test_loss.item()
# Test for label 1, 4, 5 only
label_145 = [1,4,5]
correct_145, total_145 = 0,0
# for i in range(16):
for i in label_145:
    try:    
        correct_145 += label_correct[i]
        total_145 += label_total[i]
        print('Accuracy of label {0} : {1:0.0f} / {2:0.0f} -> {3:0.3f}'.format(i, label_correct[i], label_total[i], label_correct[i]/label_total[i] * 100))
    except ZeroDivisionError:
        print("Zero Division Error -> since there are no images in label folder")
print('Test Accuracy of label 1,4,5 : {0:0.3f}, {1:0.0f} / {2:0.0f}'.format(correct_145/total_145 * 100, correct_145, total_145 ) )

Data reading time spent :  0:00:00.614359
==> Testing the model..
Accuracy of label 1 : 197 / 200 -> 98.500
Accuracy of label 4 : 197 / 200 -> 98.500
Accuracy of label 5 : 193 / 200 -> 96.500
Test Accuracy of label 1,4,5 : 97.833, 587 / 600
